In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
import sys

sys.path.append("../technique")

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os

In [ ]:
from utils import *

In [ ]:
from mobilite import add_compensation, build_data, compute_result, get_df, get_results

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
a = get_results(base)
a

In [ ]:
df, compens_constant = get_df()

In [ ]:
rdf = pd.DataFrame(
    data={
        "pu_fichier": df.PU,
        "pu_calc": df.PU * 0,
        "tp": df.PU * 0,
    }
)
data, complement_df = build_data(df, rdf)
scenario = StrasbourgSurveyScenario(base, data=data)
(res, recettes) = compute_result(scenario, complement_df, 0, compens_constant)

In [ ]:
res[~res.ok].titre_fichier.unique()

In [ ]:
res[res.titre_fichier == "Emeraude"]

In [ ]:
assert [*res[~res.ok].titre_fichier.unique()] == [" +65 Annuel", "PMR Gratuit"]

In [ ]:
if False:
    denomb_df = pd.DataFrame(
        data={
            "QF": res_df.QF,
            "plein": ~res_df.reduit,
            "quantité_fichier": res_df.quantité_fichier,
        }
    )
    denomb = pd.pivot_table(
        denomb_df, index=["QF", "plein"], values="quantité_fichier", aggfunc=sum
    )
    denomb
    merge_bareme_population = denomb.quantité_fichier.cumsum()
    df[["QF", "quantité"]].groupby(by="QF").sum().cumsum()

In [ ]:
if False:
    static_sample_count = 50
    static_sample_ids = np.repeat(list(range(static_sample_count)), count)
    static_sample_individu_df = pd.DataFrame(
        {
            "sample_id": static_sample_ids,
            "famille_id": list(range(count * static_sample_count)),
            "qfrule": np.tile(np.repeat(df.QF, df.quantité), static_sample_count),
            "agerule": np.tile(np.repeat(df.AGE, df.quantité), static_sample_count),
            "taux_incapacite": np.tile(
                np.repeat(np.where(df.Titres.str.contains("PMR"), 0.8, 0), df.quantité),
                static_sample_count,
            ),
        }
    )

    determine_qf(static_sample_individu_df, unif_qf, alea_caf_fiscal)

    rr = static_sample_individu_df.groupby(
        by="sample_id"
    ).eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial.rank(
        method="first"
    )

    df[["QF", "quantité"]].groupby(by="QF").sum()

    indexes = rr.isin(df[["QF", "quantité"]].groupby(by="QF").sum().cumsum().quantité)
    rr_res = pd.DataFrame(
        data={
            "rank": rr[indexes],
            "value": static_sample_individu_df.eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial[
                indexes
            ],
        }
    )

    rr_res.groupby(by="rank").describe()

    no_reduit_indexes = rr.isin(merge_bareme_population)
    no_reduit_rr_res = pd.DataFrame(
        data={
            "rank": rr[no_reduit_indexes],
            "value": static_sample_individu_df.eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial[
                no_reduit_indexes
            ],
        }
    )

    no_reduit_rr_res_desc = no_reduit_rr_res.groupby(by="rank").value.describe()
    no_reduit_rr_res_desc

    no_reduit_rr_res_desc["50%"].round().astype("int64")